# Class for Benchmark Step

We have developed a python class that implements the complete benchmark step procedure for **QPE** for a $R_z^n(\vec{\theta})$ operator for a fixed number of: *qubits*, *auxiliary number of qubits* and a *type of angle selection*: the *QPE_RZ* class from the *QPE.qpe_rz* module.

This notebook shows how this class works

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
sys.path.append('../../')
%matplotlib inline

## 0. QPU

See notebook 01_BTC_03_QPE_for_rzn_rz_library.ipynb

In [ ]:
# myQLM qpus
sys.path.append("../../../")
from qpu.select_qpu import select_qpu
# List with the strings that should be provided for an ideal QPU
ideal_qpus = ["c", "python", "linalg", "mps", "qlmass_linalg", "qlmass_mps"]
qpu_config = {
    "qpu_type": ideal_qpus[0], 
}
qpu = select_qpu(qpu_config)

## 1. The *QPE_RZ* class

We need to initialize the class. This should be done with a Python dictionary that configures the step benchmark execution. Mandatory keywords for the dictionary are:

* **number_of_qbits**: the number of qubits for applying the Kronecker products of $R_z$
* **auxiliar_qbits_number**: auxiliary number of qubits for performing **QPE**
* **angles**: the angles for the $R_z^n(\vec{\theta})$ operator. It can be:
    * float number: the same angle to all $R_z$
    * list with angles: Each angle will be applied to a $R_z$. The number of elements MUST be equal to the number of qubits.
    * string: Two different strings can be provided:
        * *random*: random angles will be provided to each $R_z$
        * *exact*: In this case, random angles will be provided to each $R_z$ but the final result will have a precision related with the number of auxiliary number of qubits for the **QPE**


In [ ]:
from QPE.qpe_rz import QPE_RZ

In [ ]:
n_qbits = 5
# Fix the precision of the QPE
aux_qbits = 6
# angles
angles = [np.pi / 2.0 for i in range(n_qbits)]
# Dictionary for configuring the class
qpe_dict = {
    'number_of_qbits': n_qbits,
    'auxiliar_qbits_number': aux_qbits,
    'angles': angles,
    #'angles' : 'random',    
    'qpu': qpu,
    'shots': 0
}

In [ ]:
qpe_rz_b = QPE_RZ(**qpe_dict)
qpe_rz_b.exe()

When the computation is performed (*exe* method) 2 different metrics are computed:

* Kolmogorov-Smirnov distance (**ks** attribute): it is a metric for comparing probability distributions. If it is near zero then the 2 distributions are very similar.
* Fidelity: (**fidelity** attribute): assumes that the 2 obtained eigenvalues distribution (the theoretical and the quantum one) are vectors and computes the cosine of the angle between them. For **fidelity** near one then the vectors are more equal. 

Meanwhile, the **Kolmogorov-Smirnov** is the good metric for any configuration of **QPE** of the $R_z^{\otimes n}(\vec{\theta})$ operator. The **fidelity** gives a very good metric when the configuration is such that the eigenvalues can be obtained exactly with the **QPE**.

In the following sections we use som examples for comparing the 2 metrics behaviour.


In [ ]:
print(qpe_rz_b.ks, qpe_rz_b.fidelity)

The class provide access to other attributes that can be useful like:

* *theorical_eigv_dist*: pandas DataFrame with the theoretical eigenvalue distribution
* *quantum_eigv_dist*: pandas DataFrame with the QPE eigenvalue distribution

In [ ]:
qpe_rz_b.theorical_eigv_dist

In [ ]:
qpe_rz_b.quantum_eigv_dist

### General QPE configuration

Here we are going to execute the **QPE** of the $R_z^{\otimes n}(\vec{\theta})$ operator for a random selection  of the angles. In this case, the eigenvalues of the operator can not be obtained exactly using  the QPE (we would need an infinite number of qubits to have the correct precision for obtaining exactly the eigenvalues). In this case, the **fidelity** metric does not work properly, even in the exact simulation execution, but the **Kolmogorov-Smirnov** will work.

For this case, we set the variable angles to *random* for selecting random angles for each $R_z$

In [ ]:
#Random Angles
n_qbits = 5
# Fix the precision of the QPE
aux_qbits = 10
# angles
# Dictionary for configuring the class
qpe_dict = {
    'number_of_qbits' : n_qbits,
    'auxiliar_qbits_number' : aux_qbits,
    #'angles' : angles,
    'angles' : 'random',    
    'qpu' : qpu,
    #'shots' : 0
}

In [ ]:
qpe_rz_b = QPE_RZ(**qpe_dict)
qpe_rz_b.exe()

In [ ]:
print(qpe_rz_b.ks, qpe_rz_b.fidelity)

As can be seen the **Kolmogorov-Smirnov** will give us a value near of 0, but the **fidelity** is far away from zero. We can plot the distributions to compare the results

In [ ]:
plt.plot(qpe_rz_b.theorical_eigv_dist['lambda'], qpe_rz_b.theorical_eigv_dist['Probability'], '-')
plt.plot(qpe_rz_b.quantum_eigv_dist['lambda'], qpe_rz_b.quantum_eigv_dist['Probability'], '-o')
plt.xlabel('$\lambda$')
plt.ylabel('Probability')
plt.legend(['Theorical', 'QPE'])

The cumulative distributions give us a more exact view of the similaritary of the distributions

In [ ]:
%matplotlib inline

In [ ]:
plt.plot(qpe_rz_b.theorical_eigv_dist['lambda'], qpe_rz_b.theorical_eigv_dist['Probability'].cumsum(), 'o')
plt.plot(qpe_rz_b.quantum_eigv_dist['lambda'], qpe_rz_b.quantum_eigv_dist['Probability'].cumsum(), '-')
plt.xlabel('$\lambda$')
plt.ylabel('Probability')
plt.legend(['Theorical', 'QPE'])
plt.show()

### Particular configuration

We can fix the angles of the $R_z^{\otimes n}(\vec{\theta})$ operator in such a way that the **QPE** can computes exactly the eigenvalues. In this case the **fidelity** works in a very accurate way. This particular configuration is very useful for assesing the capabilities of a quantum computer for executing **QPE** without the noise that have the general case.

For an exact computation we have to provide to the different angles of the $R_z$'s angles that should be multiple of the minimum angular precision of the **QPE** that will be given by: 

$$\delta \theta = \frac{4 \pi}{2^{m}}$$

where $m$ is the number of auxiliar qbuits of the **QPE**.

Providing the *exact* to the keyword **angles** the class compute the angles of the different $R_z$'s by using:

$$\theta_{i+1} = \theta_i +  a * \delta \theta$$ 

with $\theta_0 = \frac{\pi}{2}$ and $a$ random variable that can be $\{-1,1\}$

In [ ]:
n_qbits = 7
# Fix the precision of the QPE
aux_qbits = 7
# angles. The eigenvalues will be sum of the minimum preccision of the QPE
#angles = [4 * np.pi / 2 ** aux_qbits for i in range(n_qbits)]
angles = 'exact'
# Dictionary for configuring the class
qpe_dict = {
    'number_of_qbits' : n_qbits,
    'auxiliar_qbits_number' : aux_qbits,
    'angles' : angles,
    #'angles' : 'random',    
    'qpu' : qpu,
    #'shots' : 0
}

In [ ]:
qpe_rz_b.angles

In [ ]:
qpe_rz_b = QPE_RZ(**qpe_dict)
qpe_rz_b.exe()

In [ ]:
print(qpe_rz_b.ks, qpe_rz_b.fidelity)

As can be seen the **fidelity** now is near to 1. This is because the **QPE**, now, allows to compute exactly the eigenvalues of the operator

In [ ]:
%matplotlib inline
plt.plot(qpe_rz_b.theorical_eigv_dist['lambda'], qpe_rz_b.theorical_eigv_dist['Probability'], '-')
plt.plot(qpe_rz_b.quantum_eigv_dist['lambda'], qpe_rz_b.quantum_eigv_dist['Probability'], '-o')
plt.xlabel('$\lambda$')
plt.ylabel('Probability')
plt.legend(['Theorical', 'QPE'])

In [ ]:
%matplotlib inline
plt.bar(
    qpe_rz_b.theorical_eigv_dist['lambda'], 
    qpe_rz_b.theorical_eigv_dist['Probability'],
    width = 1 / 2**7
)
plt.xlim(0,1)
plt.ylim(0,0.12)
plt.ylabel(r'$P^{th}_{\lambda,m}\left(\frac{j}{2^m}\right)$')
plt.xlabel(r'$\frac{j}{2^m}$')

## 3.Command line execution

The module **QPE.qpe_rz** can be executed from the command line to get a complete execution of a QPE step. The following command provides help for using it:

    python qpe_rz.py -h
    
    usage: qpe_rz.py [-h] [-n_qbits N_QBITS] [-aux_qbits AUX_QBITS] [-shots SHOTS] [-angles ANGLES] [-json_qpu JSON_QPU] [-id ID] [-repetitions REPETITIONS]
                     [-folder FOLDER_PATH] [-name BASE_NAME] [--count] [--print] [--save] [--exe]

    options:
      -h, --help            show this help message and exit
      -n_qbits N_QBITS      Number of qbits for unitary operator.
      -aux_qbits AUX_QBITS  Number of auxiliary qubits for QPE
      -shots SHOTS          Number of shots: Only valid number is 0 (for exact simulation)
      -angles ANGLES        Select the angle load method: 0->exact. 1->random
      -json_qpu JSON_QPU    JSON with the qpu configuration
      -id ID                For executing only one element of the list (select the QPU)
      -repetitions REPETITIONS
                            Number of repetitions the integral will be computed.Default: 1
      -folder FOLDER_PATH   Path for storing folder
      -name BASE_NAME       Additional name for the generated files
      --count               For counting elements on the list
      --print               For printing
      --save                For saving results
      --exe                 For executing program

The qpu configuration should be provided as a JSON file. In the folder **tnbs/qpu/** two examples of JSON files can be found:
* *tnbs/qpu/qpu_ideal.json*: This JSON configures qpus for ideal simulation.
* *tnbs/qpu/qpu_noisy.json*: This JSON configures qpus for noisy simulation (only valid if the user is connected to an EVIDEN QLM)

These JSON files allow to the user configure several qpus at the same time and the user can select which one to use. 


#### **--count** argument

The **--count** argument allows to the user know how many qpus have been configured for the corresponding JSON qpu file configuration. 

If the *QPE/qpu/qpu_ideal.json* was not modified then the following command will return 6 (because 6 different qpus are configured originally in the JSON file):

    python qpe_rz.py -n_qbits 4 -aux_qbits 8 -angles 0 -json_qpu ../../qpu/qpu_ideal.json  --count

#### **--print** argument

The **--print** argument in combination with the -id ID argument allows to the user know what is the configuration of the QPU, and the configuration of the **QPE** and the $R_z^{\otimes n}(\vec{\theta})$ operator.

If the *PL/qpu/qpu_ideal.json* was not modified then the following command:

        python qpe_rz.py -n_qbits 4 -aux_qbits 8 -angles 0 -json_qpu ../../qpu/qpu_ideal.json -id 0 --print

will return:

    ##### QPE configuration #####

    {'number_of_qbits': 4, 'auxiliar_qbits_number': 8, 'shots': None, 'angles': 'exact'}

    ##### QPU configuration #####

    {'qpu_type': 'c', 't_gate_1qb': None, 't_gate_2qbs': None, 't_readout': None, 'depol_channel': {'active': False, 'error_gate_1qb': None, 'error_gate_2qbs': None}, 'idle': {'amplitude_damping': False, 'dephasing_channel': False, 't1': None, 't2': None}, 'meas': {'active': False, 'readout_error': None}}


Meanwhile the command:

    python qpe_rz.py -n_qbits 4 -aux_qbits 8 -angles 0 -json_qpu ../../qpu/qpu_ideal.json -id 3 --print
    
will return:

    ##### QPE configuration #####
    {'number_of_qbits': 4, 'auxiliar_qbits_number': 8, 'shots': None, 'angles': 'exact'}
    ##### QPU configuration #####
    {'qpu_type': 'mps', 't_gate_1qb': None, 't_gate_2qbs': None, 't_readout': None, 'depol_channel': {'active': False, 'error_gate_1qb': None, 'error_gate_2qbs': None}, 'idle': {'amplitude_damping': False, 'dephasing_channel': False, 't1': None, 't2': None}, 'meas': {'active': False, 'readout_error': None}}

#### **--exe** argument

The **--exe** argument in combination with the -id ID argument allows to the user execute the *ID* configuration case.


##### Examples


For computing the case for the 4 qubits operator with 8 auxiliary qubits for the exact angles case using the **C** linear algebra QPU the following command can be executed:

    python qpe_rz.py -n_qbits 4 -aux_qbits 8 -angles 0 -json_qpu ../../qpu/qpu_ideal.json -id 0 --exe

For computing the case for the 4 qubits operator with 8 auxiliary qubits for the random angles case using the **C** linear algebra QPU the following command can be executed:

    python qpe_rz.py -n_qbits 4 -aux_qbits 8 -angles 1 -json_qpu ../../qpu/qpu_ideal.json -id 0 --exe
    
   

## 4. my\_benchmark\_execution

A complete benchmark execution following the **TNBS** guidelines can be performed by using the **my\_benchmark\_execution.py** module in the **BTC_03_QPE** folder.


The configuration of the **BTC** can be done at the end of the script (code under  ############## CONFIGURE THE BTC  ################### line). Here the user should provide the **JSON** file for configuring the **QPU** (*qpu_json_file* variable) used for computing the **QPE** algorithm.

If there are different possible **QPU** configurations the user must change properly the *id_qpu* variable.

For configuring the different auxiliary qubits number used for the **QPE** the variable *auxiliar_qbits_number*. 

For selecting the number of qubits to execute, the variable *list_of_qbits* should be modified (a list with different numbers of qubits should be provided).

**NOTE** 

The bechmark will be executed for each number of qubits and auxiliary qubits number. So if you provide:
* auxiliar_qbits_number : [10, 12]
* list_of_qbits: [6, 8]

then $R_z$ operator of 15 and 18 qubits will be build and the QPE with 10 and 12 auxiliary qubits (for each operator) will be executed (4 different executions will be done).

The folder for saving the generated files can be provided to the *saving_folder* variable. 

The benchmark execution can be changed by modifying the different keys and variables under the ############## CONFIGURE THE BENCHMARK EXECUTION  ################# line (for default TNBS benchmark execution these keys should not be modified).



## 5. Generating the JSON file.

Once the files from a complete benchmark execution are generated the information should be formated following the **NEASQC JSON schema**. For doing this the **neasqc_benchmark.py** module can be used. At the end of the file the path to the folder where all the files from benchmark are stored should be provided to the variable **folder**.

For creating the JSON file following command should be executed:

    python neasqc_benchmark.py

## 6. Complete Workflow.

The bash script **benchmark_exe.sh** allows automatize the execution of the benchmark and the JSON file generation (once the *my_benchmark_execution.py* and the *neasqc_benchmark.py* are properly configured).

    bash benchmark_exe.sh